In [2]:
cd C:\Users\CFILT\Desktop\dataquest\Machine Learning Fundamentals

C:\Users\CFILT\Desktop\dataquest\Machine Learning Fundamentals


In [5]:
import numpy as np
import pandas as pd

dc_listings = pd.read_csv("dc_airbnb.csv")
stripped_commas = dc_listings['price'].str.replace(',', '')
stripped_dollars = stripped_commas.str.replace('$', '')
dc_listings['price'] = stripped_dollars.astype('float')
dc_listings=dc_listings.iloc[np.random.permutation(dc_listings.shape[0])]
split_one=dc_listings[:1862]
split_two=dc_listings[1862:]

In [6]:
## 2. Holdout Validation ##

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

train_one = split_one
test_one = split_two
train_two = split_two
test_two = split_one
knn=KNeighborsRegressor(algorithm='auto',n_neighbors=5)
knn.fit(train_one[["accommodates"]],train_one["price"])
p1=knn.predict(test_one[["accommodates"]])
iteration_one_rmse=(mean_squared_error(test_one["price"],p1))**(1/2)
knn=KNeighborsRegressor(n_neighbors=5)
knn.fit(train_two[["accommodates"]],train_two["price"])
p2=knn.predict(test_two[["accommodates"]])
iteration_two_rmse=(mean_squared_error(test_two["price"],p2))**(1/2)

avg_rmse=(iteration_one_rmse+iteration_two_rmse)/2


In [7]:
## 3. K-Fold Cross Validation ##

dc_listings.set_value(dc_listings.index[0:744],"fold",1)
dc_listings.set_value(dc_listings.index[744:1488],"fold",2)
dc_listings.set_value(dc_listings.index[1488:2232],"fold",3)
dc_listings.set_value(dc_listings.index[2232:2976],"fold",4)
dc_listings.set_value(dc_listings.index[2976:3723],"fold",5)


,host_response_rate,host_acceptance_rate,host_listings_count,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews,latitude,longitude,city,zipcode,state,fold
2705,100%,100%,4,9,Entire home/apt,3.0,2.5,6.0,399.0,$99.00,$249.00,1,45,24,38.913848,-77.014345,Washington,20001,DC,1.0
3348,88%,100%,1,4,Entire home/apt,2.0,2.5,3.0,170.0,$100.00,$400.00,3,9,1,38.932213,-77.072802,Washington,20016,DC,1.0
1754,60%,85%,1,2,Entire home/apt,0.0,1.0,1.0,85.0,$15.00,NaN,1,20,13,38.950572,-77.065636,Washington,20008,DC,1.0
2250,100%,88%,6,2,Private room,1.0,1.5,1.0,79.0,NaN,$150.00,6,1125,8,38.897249,-76.985674,Washington,20002,DC,1.0
1057,100%,100%,3,2,Entire home/apt,0.0,1.0,1.0,160.0,$75.00,$500.00,2,1125,26,38.908195,-77.047650,Washington,20037,DC,1.0
2828,67%,0%,2,4,Entire home/apt,3.0,1.0,3.0,250.0,$100.00,$500.00,5,1125,0,38.928215,-77.039629,Washington,20010,DC,1.0
1605,NaN,NaN,1,2,Private room,1.0,1.5,1.0,75.0,$25.00,$95.00,2,1125,0,38.949349,-77.023450,Washington,20011,DC,1.0
1863,NaN,NaN,1,1,Private room,1.0,1.0,1.0,75.0,NaN,NaN,2,14,0,38.952382,-77.010213,Washington,20011,DC,1.0
2670,88%,40%,1,6,Entire home/apt,3.0,2.5,3.0,165.0,$100.00,$300.00,1,1125,0,38.926654,-77.007928,Washington,20002,DC,1.0
3435,100%,100%,1,2,Private room,1.0,1.0,1.0,75.0,$10.00,NaN,1,90,95,38.884173,-76.981130,Washington,20003,DC,1.0


In [8]:
## 4. First iteration ##

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
train_one=dc_listings[dc_listings["fold"]!=1]
test_one=dc_listings[dc_listings["fold"]==1]
knn=KNeighborsRegressor(algorithm='auto',n_neighbors=5)
knn.fit(train_one[["accommodates"]],train_one["price"])
labels=knn.predict(test_one[["accommodates"]])
iteration_one_rmse=(mean_squared_error(test_one["price"],labels))**(1/2)

In [9]:
## 5. Function for training models ##

# Use np.mean to calculate the mean.
import numpy as np
fold_ids = [1,2,3,4,5]
def train_and_validate(df,folds):
    rmses = []
    for val in folds:
        train_one=dc_listings[dc_listings["fold"]!=val]
        test_one=dc_listings[dc_listings["fold"]==val]
        knn=KNeighborsRegressor(algorithm='auto',n_neighbors=5)
        knn.fit(train_one[["accommodates"]],train_one["price"])
        labels=knn.predict(test_one[["accommodates"]])
        iteration_one_rmse=mean_squared_error(test_one["price"],labels)**(1/2)
        rmses.append(iteration_one_rmse)
    return rmses

rmses=train_and_validate(dc_listings,fold_ids)
avg_rmse=np.mean(rmses)

In [10]:

## 6. Performing K-Fold Cross Validation Using Scikit-Learn ##

from sklearn.model_selection import cross_val_score, KFold
kf = KFold(5, shuffle=True, random_state=1)
knn=KNeighborsRegressor()
mses=cross_val_score(knn,dc_listings[["accommodates"]],dc_listings["price"], scoring='neg_mean_squared_error', cv=kf)
rmses=[abs(x)**(1/2) for x in mses]
avg_rmse=sum(rmses)/len(rmses)


In [11]:
from sklearn.model_selection import cross_val_score, KFold

num_folds = [3, 5, 7, 9, 10, 11, 13, 15, 17, 19, 21, 23]

for fold in num_folds:
    kf = KFold(fold, shuffle=True, random_state=1)
    model = KNeighborsRegressor()
    mses = cross_val_score(model, dc_listings[["accommodates"]], dc_listings["price"], scoring="neg_mean_squared_error", cv=kf)
    rmses = np.sqrt(np.absolute(mses))
    avg_rmse = np.mean(rmses)
    std_rmse = np.std(rmses)
    print(str(fold), "folds: ", "avg RMSE: ", str(avg_rmse), "std RMSE: ", str(std_rmse))

3 folds:  avg RMSE:  139.96572532 std RMSE:  4.31347930673
5 folds:  avg RMSE:  125.456805652 std RMSE:  29.7732074343
7 folds:  avg RMSE:  124.340568092 std RMSE:  30.1593111096
9 folds:  avg RMSE:  129.464134064 std RMSE:  27.473884123
10 folds:  avg RMSE:  127.84502242 std RMSE:  31.9937463836
11 folds:  avg RMSE:  129.61925829 std RMSE:  31.3823023158
13 folds:  avg RMSE:  127.070750843 std RMSE:  34.899684106
15 folds:  avg RMSE:  136.2903675 std RMSE:  37.4441170651
17 folds:  avg RMSE:  129.425319336 std RMSE:  37.6258904667
19 folds:  avg RMSE:  127.397574632 std RMSE:  41.0093627886
21 folds:  avg RMSE:  128.670960488 std RMSE:  43.216341841
23 folds:  avg RMSE:  125.886077288 std RMSE:  46.647985042
